1. The following Python script generates three lists (one for each Payer - Aetna, Horizon, and Humana) of non attributed members with PCP appointments.
2. The appointment data is obtained from Athena Database.
3. The final lists are saved in the Output folder as : "Engd Members with No Attribution Jan2023"

In [12]:
#import the dependencies
import pandas as pd
import sqlalchemy 
from sqlalchemy import create_engine
import pyodbc
import numpy as np

In [13]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'ODBC Driver 17 for SQL Server',
 'ODBC Driver 13 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)']

In [14]:
#connecting with Athena for appointment data
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=cl1d-bisql01.citymd.local;"
            "Database=Summit;"
            "Trusted_Connection=yes;")
cnxn = pyodbc.connect(cnxn_str)
cnxn.autocommit=True
cursor =cnxn.cursor()

In [15]:
#get the insurance of all the members
query =" \
select a.patientid,a.firstname,a.lastname,a.dob,a.sex,a.insurancepackageid,eligibilityservicedate, \
                      eligibilitystatus,SequenceNumber,\
                      row_number() over (partition by patientid order by eligibilityservicedate desc) as rn \
                      into ##temp1 \
                      from [Summit].[dbo].[Athena_PatientInsurance] a \
                      where  eligibilitystatus = 'Y' AND SequenceNumber = 1"
                                
cursor.execute(query)

In [16]:
#store the latest insurance of the member in a temp table
cnxn = pyodbc.connect(cnxn_str)
cnxn.autocommit=True
cursor =cnxn.cursor()
sql = "Select * into ##temp3 from ##temp1 where rn = 1"
cursor.execute(sql)

In [17]:
#cnxn = pyodbc.connect(cnxn_str)
#cnxn.autocommit=True
#cursor =cnxn.cursor()
#sql= "Select * into ##temp2 from ##temp1 where rn = 1"
#cursor.execute(sql)

In [19]:
Payer_list = ["'AETNA MEDICARE'",
              "'BCBS HORIZON'",
              "'UNITED HEALTHCARE'"]

Payer_Roster = ['Roster/No Engagement AetnaList Jan2023.xlsx' ,
                'Roster/No Engagement HorizonList Jan2023.xlsx', 
                'Roster/No Engagement UnitedList Jan2023.xlsx']

ofolder = ["U:/Actuary/Contestation/Aetna/Engaged Members with No Attribution/Output/",  
           "U:/Actuary/Contestation/Horizon/Engaged Members with No Attribution/Output/",
          "U:/Actuary/Contestation/United/Engaged Members with No Attribution/Output/"]

#Change the name of the file every month
File_name = "Engd Members with No Attribution Jan2023(f).xlsx"


for Payer in Payer_list:
    cnxn = pyodbc.connect(cnxn_str)
    cnxn.autocommit=True
    cursor =cnxn.cursor()
    
    #get the latest PCP appointment informaiton
    sql ="""select distinct cast (a.patientid as INT) as mrn,
            a.firstname,a.lastname,a.dob,a.sex,b.custominsurancegrouping,a.eligibilityservicedate, 
            eligibilitystatus,rn,
            SequenceNumber,
            Planpayer,
            d.serviceline,d.engagement, 
            d.lastvisitdate,LastPCvisitDate,PCP_Reason_Detail,Attributed_YN 
            from ##temp3 a 
            join [Summit].[dbo].[Athena_Payer] b on a.insurancepackageid = b. insurancepackageid 
            left join VBC.dbo.VBCAttributions c on a.patientid = c.enterpriseid 
            left join VBC.dbo.PatSummitProviderEngagement d on a.patientid = d.enterpriseid 
            where b.custominsurancegrouping = {}
            AND lastname <> 'TESTPATIENT'
            AND eligibilitystatus = 'Y' 
            AND SequenceNumber = 1
            order by mrn""".format(Payer)
    
    if Payer == Payer_list[0]:
        Athena_appts = pd.read_sql(sql,cnxn)
        
        #upload the Roster file
        roster = pd.read_excel(Payer_Roster[0],sheet_name= 'Engagement_List')
        
        #merge the athena_appt table with the Roster to Flag the attribution
        final_df = pd.merge(Athena_appts,roster[['mrn','memberNo','Effective Date','Risk Score (Projected)']], how='left' ,on='mrn')
        final_df = final_df.replace(np.nan,'-')
        
        #filter the df to include only the members with engaged PCP appointments
        final_df = final_df[(final_df['serviceline'] == 'PC') & (final_df['engagement'] == 'Engaged' )]
        
        #export the final_df to an excel sheet
        final_df.to_excel(ofolder[0]+File_name,index=False)
    
    elif Payer == Payer_list[1]:
        Athena_appts = pd.read_sql(sql,cnxn)
        
        #upload the Roster file
        print(Payer_Roster[1])
        roster = pd.read_excel(Payer_Roster[1],sheet_name= 'Engagement_List')
        
        #merge the athena_appt table with the Roster to Flag the attribution
        final_df = pd.merge(Athena_appts,roster[['mrn','memberNo','RUNDATE','RISK_SCORE']], how='left' ,on='mrn')
        final_df = final_df.replace(np.nan,'-')

        #filter the df to include only the members with engaged PCP appointments
        final_df = final_df[(final_df['serviceline'] == 'PC') & (final_df['engagement'] == 'Engaged' )]
        
        #export the final_df to an excel sheet
        final_df.to_excel(ofolder[1] +File_name,index=False)
        print(ofolder[1])
        
    elif Payer == Payer_list[2]:
        Athena_appts = pd.read_sql(sql,cnxn)

        #upload the Roster file
        roster = pd.read_excel(Payer_Roster[2],sheet_name= 'Engagement_List')
        
        #merge the athena_appt table with the Roster to Flag the attribution
        final_df = pd.merge(Athena_appts,roster[['mrn','memberNo','Effective Date']], how='left' ,on='mrn')
        final_df = final_df.replace(np.nan,'-')

        #filter the df to include only the members with engaged PCP appointments
        final_df = final_df[(final_df['serviceline'] == 'PC') & (final_df['engagement'] == 'Engaged' )]
        
        #export the final_df to an excel sheet
        final_df.to_excel(ofolder[2] +File_name,index=False)
 
    else:
        print('dint work')
print('COMPLETED')




C:\Users\krao2\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\krao2\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Roster/No Engagement HorizonList Jan2023.xlsx
U:/Actuary/Contestation/Horizon/Engaged Members with No Attribution/Output/


C:\Users\krao2\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


COMPLETED


In [ ]:
#roster = pd.read_excel("Roster/No Engagement UnitedList Jan2023.xlsx",sheet_name= 'Engagement_List')
#roster